<a href="https://colab.research.google.com/github/anil2k/groundingdino-auto-annotation/blob/main/automatic_annotation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object detection using Grounding DINO

In [1]:
%cd /content/drive/MyDrive/Automatic_Annotation

/content/drive/MyDrive/Automatic_Annotation


In [2]:
!git clone https://github.com/IDEA-Research/GroundingDINO.git

fatal: destination path 'GroundingDINO' already exists and is not an empty directory.


In [3]:
cd GroundingDINO/

/content/drive/MyDrive/Automatic_Annotation/GroundingDINO


In [ ]:
pip install -e .

In [5]:
!mkdir /content/drive/MyDrive/Automatic_Annotation/GroundingDINO/weights
%cd /content/drive/MyDrive/Automatic_Annotation/GroundingDINO/weights
!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

mkdir: cannot create directory ‘/content/drive/MyDrive/Automatic_Annotation/GroundingDINO/weights’: File exists
/content/drive/MyDrive/Automatic_Annotation/GroundingDINO/weights


In [6]:
import os
print(os.getcwd())

/content/drive/MyDrive/Automatic_Annotation/GroundingDINO/weights


In [7]:
ls

groundingdino_swint_ogc.pth  groundingdino_swint_ogc.pth.1


In [8]:
%cd /content/drive/MyDrive/Automatic_Annotation/GroundingDINO

/content/drive/MyDrive/Automatic_Annotation/GroundingDINO


In [9]:
from groundingdino.util.inference import load_model, load_image, predict, annotate
import cv2
config_path="/content/drive/MyDrive/Automatic_Annotation/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py"
weights_path="/content/drive/MyDrive/Automatic_Annotation/GroundingDINO/weights/groundingdino_swint_ogc.pth"

model = load_model(config_path, weights_path)
IMAGE_PATH = "/content/drive/MyDrive/Automatic_Annotation/test_images/ppe.jpg"
TEXT_PROMPT = "Reflective Jacket . saftey helmet . person . goggle"
BOX_TRESHOLD = 0.35
TEXT_TRESHOLD = 0.25

image_source, image = load_image(IMAGE_PATH)

boxes, logits, phrases = predict(
    model=model,
    image=image,
    caption=TEXT_PROMPT,
    box_threshold=BOX_TRESHOLD,
    text_threshold=TEXT_TRESHOLD
)

annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)
print("boxes: {}, logits: {}, phrases: {}".format(boxes, logits, phrases))

cv2.imwrite("annotated_image.jpg", annotated_frame)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:905: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


boxes: tensor([[0.6505, 0.5618, 0.3939, 0.8713],
        [0.4397, 0.2458, 0.0878, 0.0987],
        [0.3682, 0.5476, 0.3150, 0.8989],
        [0.3673, 0.6041, 0.3101, 0.6026],
        [0.4458, 0.1755, 0.1138, 0.1546],
        [0.6785, 0.6834, 0.3351, 0.6139],
        [0.6818, 0.2070, 0.1414, 0.1621],
        [0.6782, 0.2856, 0.1041, 0.0574]]), logits: tensor([0.7797, 0.7589, 0.7525, 0.6081, 0.6669, 0.5514, 0.4242, 0.5307]), phrases: ['person', 'goggle', 'person', 'reflective jacket', 'saftey helmet', 'reflective jacket', 'saftey helmet', 'goggle']


True

# Automate annotation process

In [2]:
%cd /content/drive/MyDrive/Automatic_Annotation
%cd GroundingDINO/

/content/drive/MyDrive/Automatic_Annotation
/content/drive/MyDrive/Automatic_Annotation/GroundingDINO


In [ ]:
pip install -e .

In [ ]:
import os
import typer
import cv2
import torch
import torchvision
from groundingdino.util.inference import load_model, load_image, predict, annotate
from tqdm import tqdm

text_prompt = 'reflective jacket, helmet, goggle, person'
box_threshold = 0.35
text_threshold = 0.10
export_annotated_images = True
config_path="/content/drive/MyDrive/Automatic_Annotation/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py"
weights_path="/content/drive/MyDrive/Automatic_Annotation/GroundingDINO/weights/groundingdino_swint_ogc.pth"
# model = load_model(config_path, weights_path)

In [ ]:
import os
import xml.etree.ElementTree as ET
import xml.dom.minidom
import cv2
import torchvision.ops
from tqdm import tqdm
import supervision as sv

image_directory = '/content/drive/MyDrive/Automatic_Annotation/test_images'
dataset = os.listdir(image_directory)

# Define a function to create Pascal VOC format XML annotation
def create_pascal_voc_xml(image_filename, image_shape, boxes, labels):
    annotation = ET.Element("annotation")

    folder = ET.SubElement(annotation, "folder")
    folder.text = "your_folder_name"  # Replace with the folder name where the images are located

    filename = ET.SubElement(annotation, "filename")
    filename.text = image_filename

    source = ET.SubElement(annotation, "source")
    database = ET.SubElement(source, "database")
    database.text = "Unknown"

    size = ET.SubElement(annotation, "size")
    width = ET.SubElement(size, "width")
    height = ET.SubElement(size, "height")
    depth = ET.SubElement(size, "depth")

    width.text = str(image_shape[1])
    height.text = str(image_shape[0])
    depth.text = str(image_shape[2])

    segmented = ET.SubElement(annotation, "segmented")
    segmented.text = "0"

    for box, label in zip(boxes, labels):
        object = ET.SubElement(annotation, "object")
        name = ET.SubElement(object, "name")
        pose = ET.SubElement(object, "pose")
        truncated = ET.SubElement(object, "truncated")
        difficult = ET.SubElement(object, "difficult")
        bndbox = ET.SubElement(object, "bndbox")
        xmin = ET.SubElement(bndbox, "xmin")
        ymin = ET.SubElement(bndbox, "ymin")
        xmax = ET.SubElement(bndbox, "xmax")
        ymax = ET.SubElement(bndbox, "ymax")

        name.text = label
        pose.text = "Unspecified"
        truncated.text = "0"
        difficult.text = "0"
        xmin.text = str(int(box[0]))
        ymin.text = str(int(box[1]))
        xmax.text = str(int(box[2]))
        ymax.text = str(int(box[3]))

    # return ET.tostring(annotation, encoding="unicode")

    # Format the XML for better readability
    xml_string = xml.dom.minidom.parseString(ET.tostring(annotation)).toprettyxml()

    return xml_string

# Initialize a directory for saving Pascal VOC XML annotations
voc_annotation_dir = os.path.join("/content/drive/MyDrive/Automatic_Annotation","voc_annotations")
os.makedirs(voc_annotation_dir, exist_ok=True)

# Iterate through your dataset
for image_filename in tqdm(dataset):
    image_path = os.path.join(image_directory, image_filename)
    image_source, image = load_image(image_path)
    boxes, logits, phrases = predict(
        model=model,
        image=image,
        caption=text_prompt,
        box_threshold=box_threshold,
        text_threshold=text_threshold,
        device="cpu"
    )

    c,h, w = image.shape
    print("height width channel : ",h, w, c)
    boxes_rs = boxes * torch.Tensor([w, h, w, h])

    rel_box = torchvision.ops.box_convert(boxes=boxes_rs, in_fmt="cxcywh", out_fmt="xyxy").numpy()

    # Create the Pascal VOC XML annotation for this image
    voc_xml_annotation = create_pascal_voc_xml(image_filename=image_filename,image_shape= [h,w,c], boxes=rel_box, labels=phrases)

    # Save the Pascal VOC XML annotation to a file
    voc_xml_filename = os.path.join(voc_annotation_dir, f"{os.path.splitext(image_filename)[0]}.xml")
    print("voc_xml_filename",voc_xml_filename)
    with open(voc_xml_filename, "w") as voc_xml_file:
        voc_xml_file.write(voc_xml_annotation)

    # Save the annotated image with bounding boxes
    annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)
    output_file = image_path[:-4] + "_annotated_image.jpg"
    cv2.imwrite(output_file, annotated_frame)

print(f"Pascal VOC annotations saved to {voc_annotation_dir}")